<a href="https://colab.research.google.com/github/mamekin05108/signatecup2024summer/blob/bra1/lightcat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 10.0 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import re
import lightgbm as lgb
from sklearn import metrics
from sklearn.metrics import f1_score, roc_auc_score
from sklearn.model_selection import StratifiedKFold

from sklearn.calibration import calibration_curve, CalibratedClassifierCV
from sklearn.metrics import roc_auc_score
from catboost import CatBoostClassifier, Pool
from sklearn.isotonic import IsotonicRegression

from sklearn.model_selection import KFold


# warningsを非表示にする
warnings.filterwarnings("ignore")

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [4]:
df_train = pd.read_csv("/content/drive/MyDrive/Signate/2024summer/fdata/df_train_allnum.csv")
df_test = pd.read_csv("/content/drive/MyDrive/Signate/2024summer/fdata/df_test_allnum.csv")
ss = pd.read_csv("/content/drive/MyDrive/Signate/2024summer/data/sample_submit.csv", header=None)

In [5]:
df_train.head()

,id,Age,TypeofContact,DurationOfPitch,Gender,NumberOfPersonVisiting,NumberOfFollowups,NumberOfTrips,Passport,MonthlyIncome,...,PitchSatisfactionScore_5,Designation_AVP,Designation_Executive,Designation_Manager,Designation_Senior Manager,Designation_VP,MaritalStatus_未婚,MaritalStatus_独身,MaritalStatus_結婚,MaritalStatus_離婚
0,0,1.145527e+00,1,0.077180,0,-1.908669,0.361786,1.038068,1,-1.352770,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,1,1.766003e+00,0,-0.067460,0,-1.908669,0.361786,-0.699052,1,0.639729,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
2,2,7.347917e-16,1,-0.646021,1,-1.908669,-0.746633,0.459028,0,-1.032001,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,3,-1.988390e-01,1,0.511101,1,-1.908669,-0.746633,-1.278092,0,-0.388082,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
4,4,9.387011e-01,0,0.366461,1,-1.908669,-0.746633,0.459028,0,-1.292825,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [6]:
df_train.drop(["id"], axis=1, inplace=True)
df_test.drop(["id"], axis=1, inplace=True)

In [7]:
df_test.head()

,Age,TypeofContact,DurationOfPitch,Gender,NumberOfPersonVisiting,NumberOfFollowups,NumberOfTrips,Passport,MonthlyIncome,ProdTaken,...,PitchSatisfactionScore_5,Designation_AVP,Designation_Executive,Designation_Manager,Designation_Senior Manager,Designation_VP,MaritalStatus_未婚,MaritalStatus_独身,MaritalStatus_結婚,MaritalStatus_離婚
0,0.938701,1,-0.212100,0,-1.908669,0.361786,2.196148,0,1.863454,NaN,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,-0.922728,1,-0.356740,1,-1.908669,0.361786,0.459028,1,-0.742793,NaN,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2,-1.439792,1,-0.790661,1,-1.908669,0.361786,-1.278092,0,-1.272115,NaN,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,-1.853443,0,-1.079941,0,-1.908669,0.361786,-1.278092,0,-1.273769,NaN,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
4,0.214812,0,-1.079941,0,-1.908669,0.361786,-1.278092,0,-1.155267,NaN,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [8]:
target = "ProdTaken"
train_y = df_train[target]
train_x = df_train.drop(target, axis=1)

In [9]:
test_x = df_test.drop(target, axis=1)

In [10]:
train_y.shape

(3489,)

In [11]:
test_x.shape

(3489, 40)

# Lightgbmモデル


In [35]:
params_lgb = {
    'objective': 'binary',
    'metric': 'auc',
    'num_leaves': 25,
    'learning_rate': 0.05,
    'feature_fraction': 0.4,
    'bagging_fraction': 0.8,
    'bagging_freq': 3,
    'verbose': -1,
    'lambda_l1': 0.81,
    'lambda_l2': 90
}

In [36]:
lgbr =lgb.LGBMClassifier(**params_lgb)

In [57]:
auc_sum = []
preds = []
splits = 7
kf2 = KFold(n_splits = splits, shuffle = True, random_state = 12)
for fold, (train_idx, valid_idx) in enumerate(kf2.split(train_x)):
    X_tr, y_tr = train_x.iloc[train_idx, :], train_y.iloc[train_idx]
    X_val, y_val = train_x.iloc[valid_idx, :], train_y.iloc[valid_idx]

    # LightGBM モデルの設定
    model = lgb.LGBMClassifier(
        **params_lgb  # 'n_estimators' を含むすべてのパラメータを渡す
    )

    # モデルのトレーニング
    model.fit(
        X_tr, y_tr,
        eval_metric='auc',
        eval_set=[(X_val, y_val)],
        callbacks=[lgb.early_stopping(100, verbose=True)]
    )

    #y_pred = model.predict(X_val)[:, 1]
    preds_y_proba = model.predict_proba(X_val)[:, 1]
    auc = roc_auc_score(y_val, preds_y_proba)
    pred = model.predict_proba(test_x)[:, 1]
    preds.append(pred)

    print(f'Fold {fold + 1}: AUC = {auc:4f}')
    auc_sum.append(auc)

# 特徴量の重要度を出力する
importances = model.feature_importances_
column_names = train_x.columns.values
importance_list = [
    [importances[i], column_names[i]] for i in range(len(column_names))
]
importance_list.sort(reverse=True, key=lambda x: x[0])

print("Feature importances:")
for importance, feature in importance_list:
    print(f"{importance:.4f} : {feature}")

print(np.mean(auc_sum))

Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[98]	valid_0's auc: 0.840536
Fold 1: AUC = 0.840536
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[100]	valid_0's auc: 0.808488
Fold 2: AUC = 0.808488
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[100]	valid_0's auc: 0.830101
Fold 3: AUC = 0.830101
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[100]	valid_0's auc: 0.850801
Fold 4: AUC = 0.850801
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[96]	valid_0's auc: 0.818598
Fold 5: AUC = 0.818598
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[98]	valid_0's auc: 0.841248
Fold 6: AUC = 0.841248
Training until validation

In [54]:
def run_lgbm_cv_with_best_random_state(train_x, train_y, test_x, params_lgb, splits=5, random_state_range=(0, 100)):
    best_mean_auc = -np.inf
    best_random_state = None
    best_preds = None

    for random_state in range(random_state_range[0], random_state_range[1] + 1):
        preds = []
        auc_sum = []

        # KFold クロスバリデーションの設定
        kf2 = KFold(n_splits=splits, shuffle=True, random_state=random_state)

        for fold, (train_idx, valid_idx) in enumerate(kf2.split(train_x)):
            X_tr, y_tr = train_x.iloc[train_idx, :], train_y.iloc[train_idx]
            X_val, y_val = train_x.iloc[valid_idx, :], train_y.iloc[valid_idx]

            # LightGBM モデルの設定
            model = lgb.LGBMClassifier(
                **params_lgb  # 'n_estimators' を含むすべてのパラメータを渡す
            )

            # モデルのトレーニング
            model.fit(
                X_tr, y_tr,
                eval_metric='auc',
                eval_set=[(X_val, y_val)],
                callbacks=[lgb.early_stopping(100, verbose=False)]
            )

            # 予測とAUCの計算
            preds_y_proba = model.predict_proba(X_val)[:, 1]
            auc = roc_auc_score(y_val, preds_y_proba)
            pred = model.predict_proba(test_x)[:, 1]
            preds.append(pred)

            auc_sum.append(auc)

        mean_auc = np.mean(auc_sum)
        print(f"Random State {random_state}: Mean AUC = {mean_auc:.8f}")

        # ベストAUCの更新
        if mean_auc > best_mean_auc:
            best_mean_auc = mean_auc
            best_random_state = random_state
            best_preds = preds

    print(f"\nBest Random State: {best_random_state} with Mean AUC: {best_mean_auc:.8f}")

    return best_mean_auc, best_random_state

In [56]:
best_mean_auc, best_random_state = run_lgbm_cv_with_best_random_state(
    train_x, train_y, test_x, params_lgb, splits=5, random_state_range=(0, 100)
)

Random State 0: Mean AUC = 0.82541743
Random State 1: Mean AUC = 0.82450346
Random State 2: Mean AUC = 0.82727966
Random State 3: Mean AUC = 0.82868971
Random State 4: Mean AUC = 0.82986562
Random State 5: Mean AUC = 0.82744976
Random State 6: Mean AUC = 0.82977016
Random State 7: Mean AUC = 0.83069444
Random State 8: Mean AUC = 0.82934222
Random State 9: Mean AUC = 0.82493406
Random State 10: Mean AUC = 0.82782634
Random State 11: Mean AUC = 0.82857405
Random State 12: Mean AUC = 0.83241973
Random State 13: Mean AUC = 0.82818532
Random State 14: Mean AUC = 0.82686077
Random State 15: Mean AUC = 0.82807178
Random State 16: Mean AUC = 0.82893653
Random State 17: Mean AUC = 0.82968389
Random State 18: Mean AUC = 0.83030858
Random State 19: Mean AUC = 0.82801483
Random State 20: Mean AUC = 0.82859651
Random State 21: Mean AUC = 0.83039266
Random State 22: Mean AUC = 0.83043964
Random State 23: Mean AUC = 0.82741929
Random State 24: Mean AUC = 0.83073586
Random State 25: Mean AUC = 0.82794

In [58]:
last_pred_1 = 3489 * [0.0]

#１層のモデルとして使うのでpreds_
for i in range(3489):
    for j in range(splits):
        last_pred_1[i] += preds[j][i]
    last_pred_1[i] /= splits

ensemble_preds = []
ensemble_preds.append(last_pred_1)

In [59]:
count=0

In [60]:
from datetime import datetime
import pytz
# カウント変数をインクリメント
count += 1
# 日本時間を取得
japan_tz = pytz.timezone('Asia/Tokyo')
now = datetime.now(japan_tz)
timestamp = now.strftime("%Y%m%d_%H%M%S")

file_name = f"/content/drive/MyDrive/Signate/2024summer/light_{timestamp}_{count:03d}.csv"
ss[1] = last_pred_1
ss.to_csv(file_name, header=False, index=False)

# Catboostモデル

In [41]:
params_cat = {
    "iterations": 3000,  # n_estimatorsに相当
    "learning_rate": 0.01,
    "colsample_bylevel": 0.8,  # colsample_bytreeに相当
    "random_seed": 0,
    "verbose": 100,  # 学習の進捗を100回毎に表示
    "use_best_model": True,  # 早期停止のための設定
}

In [42]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
from catboost import CatBoostClassifier

auc_sum = []
preds = []
splits = 7
kf2 = KFold(n_splits=splits, shuffle=True, random_state=0)

for fold, (train_idx, valid_idx) in enumerate(kf2.split(train_x)):
    X_tr, y_tr = train_x.iloc[train_idx, :], train_y.iloc[train_idx]
    X_val, y_val = train_x.iloc[valid_idx, :], train_y.iloc[valid_idx]

    # CatBoost モデルの設定
    model = CatBoostClassifier(
        **params_cat,  # CatBoostのパラメータをここに渡す
        eval_metric='AUC',
        early_stopping_rounds=100
    )

    # モデルのトレーニング
    model.fit(
        X_tr, y_tr,
        eval_set=[(X_val, y_val)],
        #verbose=True
        verbose_eval=100
    )

    # 予測とAUCの計算
    preds_y_proba = model.predict_proba(X_val)[:, 1]
    auc = roc_auc_score(y_val, preds_y_proba)
    pred = model.predict_proba(test_x)[:, 1]
    preds.append(pred)

    print(f'Fold {fold + 1}: AUC = {auc:4f}')
    auc_sum.append(auc)

# 特徴量の重要度を出力する
importances = model.get_feature_importance()
column_names = train_x.columns.values
importance_list = [
    [importances[i], column_names[i]] for i in range(len(column_names))
]
importance_list.sort(reverse=True, key=lambda x: x[0])

print("Feature importances:")
for importance, feature in importance_list:
    print(f"{importance:.4f} : {feature}")

print(np.mean(auc_sum))

0:	test: 0.6980286	best: 0.6980286 (0)	total: 51.3ms	remaining: 2m 33s
100:	test: 0.8261049	best: 0.8261049 (100)	total: 348ms	remaining: 10s
200:	test: 0.8355485	best: 0.8360890 (190)	total: 610ms	remaining: 8.49s
300:	test: 0.8369793	best: 0.8375199 (294)	total: 853ms	remaining: 7.65s
400:	test: 0.8380286	best: 0.8389507 (375)	total: 1.09s	remaining: 7.05s
500:	test: 0.8395548	best: 0.8403816 (489)	total: 1.33s	remaining: 6.64s
600:	test: 0.8413355	best: 0.8416852 (592)	total: 1.59s	remaining: 6.34s
700:	test: 0.8433386	best: 0.8435930 (686)	total: 1.83s	remaining: 6.02s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.8435930048
bestIteration = 686

Shrink model to first 687 iterations.
Fold 1: AUC = 0.843593
0:	test: 0.8397558	best: 0.8397558 (0)	total: 2.53ms	remaining: 7.58s
100:	test: 0.8840332	best: 0.8854153 (98)	total: 231ms	remaining: 6.62s
200:	test: 0.8948598	best: 0.8948598 (200)	total: 492ms	remaining: 6.85s
300:	test: 0.8975582	best: 0.8981177 (292)	

In [43]:
last_pred_2 = 3489 * [0.0]

for i in range(3489):
    for j in range(splits):
        last_pred_2[i] += preds[j][i]
    last_pred_2[i] /= splits

ensemble_preds.append(last_pred_2)

In [44]:
from datetime import datetime
import pytz
# カウント変数をインクリメント
count += 1
# 日本時間を取得
japan_tz = pytz.timezone('Asia/Tokyo')
now = datetime.now(japan_tz)
timestamp = now.strftime("%Y%m%d_%H%M%S")

file_name = f"/content/drive/MyDrive/Signate/2024summer/Em_cat_{timestamp}_{count:03d}.csv"
ss[1] = last_pred_2
ss.to_csv(file_name, header=False, index=False)

アンサンブル

In [49]:
def find_permutations(n):
    perm_list = []
    current_perm = [0] * n

    while True:
        if sum(current_perm) == 10:
            perm_list.append(tuple(current_perm))

        i = n-1
        while i >= 0 and current_perm[i] == 10:
            current_perm[i] = 0
            i -= 1

        if i < 0:
            break

        current_perm[i] += 1

    return perm_list

n = 2
permutations = find_permutations(n)
last_pred = len(train_x) * [0.0]
l = []
for i in range(n):
    l.append(0.0)
best_score = 0
for perm in permutations:
    for i in range(len(train_x)):
        last_pred_list = len(train_x) * l
        last_pred_list[i] = [last_pred_1[i], last_pred_2[i]]
        last_pred[i] = np.dot(last_pred_list[i], perm) / 10

    auc_test = roc_auc_score(train_y, last_pred)
    if auc_test > best_score:
        best_score = auc_test
        best_perm = perm
    print(perm, auc_test)
print('best param is ', best_perm, ', best score is ', best_score)

(0, 10) 0.5018446238930911
(1, 9) 0.5014586180182701
(2, 8) 0.5010826994049861
(3, 7) 0.5007323351875962
(4, 6) 0.5002985829414993
(5, 5) 0.49978883999182255
(6, 4) 0.499332895770344
(7, 3) 0.4989879114257739
(8, 2) 0.4985339846566027
(9, 1) 0.4980726605623043
(10, 0) 0.497539380669041
best param is  (0, 10) , best score is  0.5018446238930911


In [46]:
 [last_pred_1[1], last_pred_2[1]]

[0.16852593069015756, 0.22664528337745277]

In [ ]:
lastにはいってるのはtestのy
じゃん